1.处理数据

In [1]:
import pandas as pd
import re

def get_title(name):
    if(pd.isnull(name)):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'
    
titles = {'mr': 1,
          'mrs': 2, 'mme': 2,
          'ms': 3, 'miss': 3, 'mlle': 3,
          'don': 4, 'sir': 4, 'jonkheer': 4,
          'major': 4, 'col': 4, 'dr': 4, 'master': 4, 'capt': 4,
          'dona': 5, 'lady': 5, 'countness': 5,
          'rev': 6}

def get_train_data(path):
    # 从CSV文件中读入数据
    data = pd.read_csv(path)
    
    # 取部分特征字段用于分类，清洗Age等数据
    data['Sex'] = data['Sex'].apply(lambda s: 1 if s == 'male' else 0)
    mean_age = data["Age"].mean()
    data.loc[data.Age.isnull(), "Age"] = mean_age
    
    data['Title'] = data['Name'].apply(lambda name: titles.get(get_title(name)))
    data['Honor'] = data['Title'].apply(lambda title: 1 if title == 4 or title == 5 else 0)
    
    data['Embarked'] = data['Embarked'].apply(lambda embarked: 1 if embarked == 'S' else embarked)
    data['Embarked'] = data['Embarked'].apply(lambda embarked: 2 if embarked == 'C' else embarked)
    data['Embarked'] = data['Embarked'].apply(lambda embarked: 3 if embarked == 'Q' else embarked)
    
    # 两种分类分别是幸存和死亡，‘Survived’字段是其中一种分类的标签，
    # 新增‘Deceased’字段表示第二段分类的标签，取值为‘Survived’字段取非
    data['Deceased'] = data['Survived'].apply(lambda s: int(not s))
    
    data = data.fillna(0)
    data.info()
    
    return data

def get_test_data(path):
    # 从CSV文件中读入数据
    data = pd.read_csv(path)
    
    # 取部分特征字段用于分类，清洗Age等数据
    data['Sex'] = data['Sex'].apply(lambda s: 1 if s == 'male' else 0)
    mean_age = data["Age"].mean()
    data.loc[data.Age.isnull(), "Age"] = mean_age
    
    data['Title'] = data['Name'].apply(lambda name: titles.get(get_title(name)))
    data['Honor'] = data['Title'].apply(lambda title: 1 if title == 4 or title == 5 else 0)
    
    data['Embarked'] = data['Embarked'].apply(lambda embarked: 1 if embarked == 'S' else embarked)
    data['Embarked'] = data['Embarked'].apply(lambda embarked: 2 if embarked == 'C' else embarked)
    data['Embarked'] = data['Embarked'].apply(lambda embarked: 3 if embarked == 'Q' else embarked)
    
    data = data.fillna(0)
    data.info()
    
    return data

In [2]:
data = get_train_data('../../../../dataset/titanic/train.csv')

dataset_X = data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Honor', 'Embarked']].as_matrix()
dataset_Y = data[['Survived']].as_matrix()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null float64
Title          891 non-null float64
Honor          891 non-null int64
Deceased       891 non-null int64
dtypes: float64(4), int64(8), object(3)
memory usage: 104.5+ KB


2.初步处理数据

In [3]:
from sklearn.model_selection import train_test_split

# 使用sklearn的train_test_split函数将标记数据切分为“训练数据集和验证集”
# 将全部标记数据随机洗牌后切分，其中验证数据占20%，由test_size参数指定
X_train, X_test, y_train, y_test = train_test_split(
    dataset_X, dataset_Y, test_size = 0.2, random_state = 42)

3.使用SkFlow建立模型

In [4]:
import tensorflow.contrib.learn as skflow
from sklearn import metrics

feature_cols = skflow.infer_real_valued_columns_from_input(X_train)
classifier = skflow.LinearClassifier(feature_columns=feature_cols, n_classes=2)
classifier.fit(X_train, y_train, steps=500)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\AkiKan\\AppData\\Local\\Temp\\tmppplio0oi', '_environment': 'local', '_save_summary_steps': 100, '_is_chief': True, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_task_type': None, '_master': '', '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_log_step_count_steps': 100, '_task_id': 0, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000283595FEF60>}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SK

LinearClassifier(params={'joint_weights': False, 'gradient_clip_norm': None, 'optimizer': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x000002835506EB70>, 'feature_columns': [_RealValuedColumn(column_name='', dimension=8, default_value=None, dtype=tf.float64, normalizer=None)]})

In [5]:
# 预测结果有些问题
y_pred = classifier.predict(X_test)
print(y_pred)

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from C:\Users\AkiKan\AppData\Local\Temp\tmppplio0oi\model.ckpt-500
<generator object _as_iterable at 0x000002835B1342B0>


In [6]:
accuracy = metrics.accuracy_score(y_test, y_test)
print("Accuracy: %f" % accuracy)

Accuracy: 1.000000


4.TFLearn

In [7]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tflearn

train_data = get_train_data('../../../../dataset/titanic/train.csv')
X = train_data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'Honor']].as_matrix()
Y = train_data[['Deceased', 'Survived']].as_matrix()

# mkdir checkpoint path
ckpt_dir = './ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
    
# 定义分类模型
x_input = tflearn.input_data([None, X.shape[1]])
y_pred = tflearn.layers.fully_connected(x_input, 2, activation='softmax')
net = tflearn.regression(y_pred)
model = tflearn.DNN(net)

# 读取模型存档
# if os.path.isfile(os.path.join(ckpt_dir, 'model.cpkt')):
#     model.load(os.path.join(ckpt_dir, 'model.ckpt'))
# 训练    
model.fit(X, Y, validation_set=0.1, n_epoch=50)
# 存储模型参数
model.save(os.path.join(ckpt_dir, 'model.ckpt'))
# 查看模型在训练集上的准确度
metric = model.evaluate(X, Y)
print('Accuracy on train set: %.9f' % metric[0])

Training Step: 649  | total loss: 0.47325 | time: 0.044s
| Adam | epoch: 050 | loss: 0.47325 -- iter: 768/801
Training Step: 650  | total loss: 0.47812 | time: 1.050s
| Adam | epoch: 050 | loss: 0.47812 | val_loss: 0.48891 -- iter: 801/801
--
INFO:tensorflow:E:\Document\administer\Python Scripts\git\DL-Test\3-EasyLearning\titannic\ckpt_dir\model.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Accuracy on train set: 0.794612795


In [8]:
# 读取测试数据，并进行预测
test_data = get_test_data('../../../../dataset/titanic/test.csv')
X = test_data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'Honor']].as_matrix()
predictions = np.argmax(model.predict(X), axis=1)

# 构建提交结果的数据结构，并将结果存为csv文件
submission = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Survived": predictions})
submission.to_csv("../../../../dataset/titanic/titanic_submission.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          418 non-null object
Embarked       418 non-null int64
Title          418 non-null int64
Honor          418 non-null int64
dtypes: float64(2), int64(8), object(3)
memory usage: 42.5+ KB
